In [23]:
!pip3 install requests
!pip3 install bs4
!pip3 install selenium
!pip3 install webdriver-manager
!pip3 install pandas

In [24]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager


In [7]:
url = "https://www.sunshineliststats.com/Employer/9/2025/?n=universityoftoronto"

response = requests.get(url)
response.status_code

200

In [14]:
html = response.text
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
classes = 'table-striped table-hover table-responsive no-margin emploeeetable'

table = soup.find('table', {'class': classes.split()})

headers = [th.text.strip() for th in table.find('thead').find_all('th')]
data = []
tbody = table.find('tbody')
rows = tbody.find_all('tr') if tbody else table.find_all('tr') 
for row in rows:
    cols = row.find_all('td')
    row_data = [col.text.strip() for col in cols]
    if row_data: 
        data.append(row_data)

df = pd.DataFrame(data, columns=headers)

df

,Salary,Name,Position
0,"$101,145.49\nearned in 2024",Aaron Forward,Standardized Patient Educator and Project Mana...
1,"$146,222.06\nearned in 2024",Aaron Reinke,"Associate Professor, Molecular Genetics\r\n ..."
2,"$174,955.02\nearned in 2024",Aaron Weisbrod,"Assistant Professor, Teaching Stream, Departme..."
3,"$221,440.98\nearned in 2024",Aaron Wheeler,Professor of Chemistry\r\n ...
4,"$184,555.02\nearned in 2024",Aarthi Ashok,"Professor, Teaching Stream of Biological Scien..."
5,"$102,244.13\nearned in 2024",Abbey Kachmar,Senior Labour Relations Consultant\r\n ...
6,"$191,362.42\nearned in 2024",Abby Goldstein,"Associate Professor of Education and Chair, De..."
7,"$169,830.48\nearned in 2024",Abdel-Khalig Ali,"Associate Professor, Teaching Stream, Near and..."
8,"$161,531.97\nearned in 2024",Abdi Aidid,Professor of Law\r\n ...
9,"$142,044.96\nearned in 2024",Abdollah Farhoodi,"Professor, Economics\r\n ..."


In [ ]:
driver = webdriver.Chrome()

driver.get(url)
driver.maximize_window()

pages = 344
data = []

for page in range(1, pages + 1):
    print(f"Processing page: {page}")
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    table = soup.find('table', {'class': classes.split()})

    headers = [th.text.strip() for th in table.find('thead').find_all('th')]
    tbody = table.find('tbody')
    rows = tbody.find_all('tr') if tbody else table.find_all('tr')
    page_data = []
    for row in rows:
        cols = row.find_all('td')
        row_data = [col.text.strip() for col in cols]
        if row_data:
            page_data.append(row_data)
    data.extend(page_data)

    if page < pages:
        try:
            next_button = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "li.PagedList-skipToNext a"))
            )
            driver.execute_script("arguments[0].click();", next_button)
            time.sleep(2)  

        except Exception as e:
            print(f"Error clicking 'Next' button: {e}")
            break

driver.quit()


In [ ]:
df = pd.DataFrame(data, columns=headers)

df